<a href="https://colab.research.google.com/github/FeDextroyer/MQTT/blob/master/Sample%20Extended.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install paho-mqtt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 KB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for paho-mqtt: filename=paho_mqtt-1.6.1-py3-none-any.whl size=62135 sha256=0ebfd6c50b0d4946adf51bab59b50046581a2c77134622829ff440020d9cb529
  Stored in directory: /root/.cache/pip/wheels/0f/90/29/db29bb8ddc98ec5f2363b959130c9ddbcf5cfdb4a00b6184dd
Successfully built paho-mqtt


In [ ]:
import paho.mqtt.client as paho             #mqtt library
import os
import json
import time
from datetime import datetime

ACCESS_TOKEN='9NQB5mlMQVFbTYtcdoiV'                 #Token of your device
broker="demo.thingsboard.io"                        #host name
port=1883                                           #data listening port

def on_publish(client,userdata,result):             #create function for callback
    print("data published to thingsboard \n")
    pass
client1= paho.Client("control1")                    #create client object
client1.on_publish = on_publish                     #assign function to callback
client1.username_pw_set(ACCESS_TOKEN)               #access token from thingsboard device
client1.connect(broker,port,keepalive=60)           #establish connection

while True:

   payload="{"
   payload+="\"Humidity\":28,"; 
   payload+="\"Temperature\":55"; 
   payload+="\"pressure\":87"; 
   payload+="\"luminosity\":20"; 
   payload+="}"
   ret= client1.publish("v1/devices/me/telemetry",payload) #topic-v1/devices/me/telemetry
   print("Please check LATEST TELEMETRY field of your device")
   print(payload);
   time.sleep(5)
  

data published to thingsboard 

Please check LATEST TELEMETRY field of your device
{"Humidity":28,"Temperature":55"pressure":87"luminosity":20}
data published to thingsboard 

Please check LATEST TELEMETRY field of your device
{"Humidity":28,"Temperature":55"pressure":87"luminosity":20}
data published to thingsboard 

Please check LATEST TELEMETRY field of your device
{"Humidity":28,"Temperature":55"pressure":87"luminosity":20}
data published to thingsboard 

Please check LATEST TELEMETRY field of your device
{"Humidity":28,"Temperature":55"pressure":87"luminosity":20}
Please check LATEST TELEMETRY field of your device
{"Humidity":28,"Temperature":55"pressure":87"luminosity":20}
Please check LATEST TELEMETRY field of your device
{"Humidity":28,"Temperature":55"pressure":87"luminosity":20}
Please check LATEST TELEMETRY field of your device
{"Humidity":28,"Temperature":55"pressure":87"luminosity":20}
Please check LATEST TELEMETRY field of your device
{"Humidity":28,"Temperature":55"press

In [3]:
!sudo ls
!#apt-get install -y software-properties-common python-software-properties
!apt-add-repository -y ppa:mosquitto-dev/mosquitto-ppa
!#sudo apt-get update
!apt-get install -y mosquitto mosquitto-clients

!pip install paho-mqtt
!#sudo apt-add-repository ppa:mosquitto-dev/mosquitto-ppa
!#sudo apt-get update
!#sudo apt-get install -y mosquitto

!apt-get install -y systemd
!systemctl enable mosquitto.service

sample_data
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [73.4 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,544 kB]
Get:13 http://ppa.launchpad.net/mosquitto-dev/mosquitto-ppa/ubuntu focal InRe

In [4]:
global badh, badt, badp, badl, maxtemp, maxhum, maxpress, maxlum
maxtemp = 0
maxhum = 0
maxpress = 0
maxlum = 0

badt = False
badh = False
badp = False
badl = False

In [8]:
%matplotlib notebook
import threading
import ssl
import sys
import paho.mqtt.client
import json
import numpy as np
import matplotlib.pyplot as plt
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

fig = plt.figure()
plt.ion()
host = fig.add_subplot(111)

par1 = host.twinx()
par2 = host.twinx()
par3 = host.twinx()
par4 = host.twinx()

host.set_xlabel("dataframe")
par1.set_ylabel("temperature")
par2.set_ylabel("pressure")
par3.set_ylabel("humidity")
par4.set_ylabel("luminocity")

color1 = plt.cm.viridis(0)
color2 = plt.cm.viridis(0.4)
color3 = plt.cm.viridis(.7)
color4 = plt.cm.viridis(.9)


# right, left, top, bottom
#par2.spines['right'].set_position(('outward', 60))      
# no x-ticks                 
#par2.xaxis.set_ticks([])
# Sometimes handy, same for xaxis
#par2.yaxis.set_ticks_position('right')



fig.patch.set_facecolor('red')
fig.show()
fig.canvas.draw()

p1Data = [0]
p2Data = [0]
p3Data = [0]
p4Data = [0]

dataTime = []
i = 0

def temperatureLimits(x):
    global maxtemp
    maxtemp = x
    return x

def pressureLimits(x):
    global maxpress
    maxpress = x
    return x

def humidityLimits(x):
    global maxhum
    maxhum = x
    return x

def luminocityLimits(x):
    global maxlum
    print("called!")
    print(maxlum)
    maxlum = x
    return x

def on_connect(client, userdata, flags, rc):
	print('connected (%s)' % client._client_id)
	client.subscribe(topic='[topic]', qos=2)

def on_message(client, userdata, message):
    global badh, badt, badp, badl, maxtemp, maxhum, maxpress, maxlum
    try:
        print("client got a message")
        j = json.loads(message.payload)
        print("client " + j)
        if(j["data"]):
            s = j["data"]
            print(s)
            
            joined = True
            try:
                 if s['joined']:
                    pass
            except:
                joined = False
                
            if(joined):
                return;
                
            try: 
                t = s['temperature']
                print(t)
                p1Data.append(t)
                badt = t > maxtemp
            except KeyError: p1Data.append(p1Data[-1])
            
            try: 
                p = s['pressure']
                print(p)
                p2Data.append(p)
                badp = p > maxpress
            except KeyError: p2Data.append(p2Data[-1])
                    
            try: 
                h = s['humidity']
                print(h)
                p3Data.append(h)
                badh = h > maxhum
            except KeyError: p3Data.append(p3Data[-1])
            
                                
            try: 
                l = s['luminocity']
                print(l)
                p4Data.append(l)
                badl = l > maxlum
            except KeyError: p4Data.append(p4Data[-1])
            
            
            dataTime.append(++i)
            
            par1.clear()
            par2.clear()
            par3.clear()
            par4.clear()
            
            
            p1, = par1.plot(p1Data, color=color1,label="temperature")
            p2, = par2.plot(p2Data, color=color2,label="pressure")
            p3, = par3.plot(p3Data, color=color3,label="humidity")
            p4, = par4.plot(p4Data, color=color4, label="luminocity")
            
            par1.spines['right'].set_position(('outward', 0))   
            par2.spines['right'].set_position(('outward', 40))   
            par3.spines['right'].set_position(('outward', 80))   
            par4.spines['right'].set_position(('outward', 120))   
            par1.yaxis.label.set_color(p1.get_color())
            par2.yaxis.label.set_color(p2.get_color())
            par3.yaxis.label.set_color(p3.get_color())
            par4.yaxis.label.set_color(p4.get_color())
            
            par1.yaxis.set_label_position('left')
            par2.yaxis.set_label_position('left')
            par3.yaxis.set_label_position('left')
            par4.yaxis.set_label_position('left')
            
            lns = [p1, p2, p3, p4]
            host.legend(handles=lns, loc='best')
            
            if not badh and not badt and not badp and not badl:
                fig.patch.set_facecolor('red')
            else :
                fig.patch.set_facecolor('white')
    
            fig.tight_layout() 
            fig.canvas.draw()

    except KeyError:
        print("bad msg")
    
    #print('------------------------------')
    #print('topic: %s' % message.topic)
    #print(message.payload.decode("utf-8"))
    #print('qos: %d' % message.qos)
    
once = True
client  = paho.mqtt.client.Client(client_id='oj', userdata=None,  transport="tcp")
def work(once, client):
    if once:
        once = False
        client.on_connect = on_connect
        client.on_message = on_message
        client.subscribe("data")
        
        
work(once, client)

#thread = threading.Thread(target=work, args=(once, client))
#thread.start()


<IPython.core.display.Javascript object>

In [9]:
interact_manual(temperatureLimits, x=100);
interact_manual(pressureLimits, x=1000);
interact_manual(humidityLimits, x=1000);
interact_manual(luminocityLimits, x=1000);

interactive(children=(IntSlider(value=100, description='x', max=300, min=-100), Button(description='Run Intera…

interactive(children=(IntSlider(value=1000, description='x', max=3000, min=-1000), Button(description='Run Int…

interactive(children=(IntSlider(value=1000, description='x', max=3000, min=-1000), Button(description='Run Int…

interactive(children=(IntSlider(value=1000, description='x', max=3000, min=-1000), Button(description='Run Int…

In [10]:
import time
 
clientSender  = paho.mqtt.client.Client(client_id='sender', userdata=None,  transport="tcp")
onceSender = True
def sender(once, client):
    if once:
        once = False
        client.on_connect = on_connect
        client.connect('127.0.0.1', 1883, 60)
        
        while True:
            
            data = {"data" : {"temperature" : 666}}
            payload=json.dumps(data)
            print("sender " +  payload)
            (rc, mid) = client.publish(topic="data", payload=payload, qos=2)
            time.sleep(1)
        
thread2 = threading.Thread(target=sender, args=(onceSender, clientSender))
thread2.start()

#sender(onceSender, clientSender)

Exception in thread Thread-10:
Traceback (most recent call last):
  File "/usr/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-10-fb4fd72014e7>", line 9, in sender
  File "/usr/local/lib/python3.9/dist-packages/paho/mqtt/client.py", line 914, in connect
    return self.reconnect()
  File "/usr/local/lib/python3.9/dist-packages/paho/mqtt/client.py", line 1044, in reconnect
    sock = self._create_socket_connection()
  File "/usr/local/lib/python3.9/dist-packages/paho/mqtt/client.py", line 3685, in _create_socket_connection
    return socket.create_connection(addr, timeout=self._connect_timeout, source_address=source)
  File "/usr/lib/python3.9/socket.py", line 844, in create_connection
    raise err
  File "/usr/lib/python3.9/socket.py", line 832, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection 

In [11]:
import json
data = {"data" : {"temperature" : 666}}
payload=json.dumps(data)
j = json.loads(payload)
j["data"]

{'temperature': 666}